# 프로젝트: 작사가 인공지능 만들기

## 데이터:
1Mb 남짓한 노래 가사 텍스트 파일

## 목표:
1. 특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가?
2. 텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가?
3. 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

## (1) 라이브러리 및 데이터 불러오기

In [14]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow as tf
import os
import re

print(tensorflow.__version__)

2.6.0


In [3]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' 
txt_list = glob.glob(txt_file_path) 
raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() 
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


In [7]:
# 문장 확인
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜀
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜀
    if idx > 15: break   # 문장 15개 미만만 확인
        
print(sentence)

It's not warm when she's away 


## (2) 데이터 전처리

In [10]:
# 입력된 문장 필터링
#     1. 소문자로 바꾸고, 양쪽 공백 지우기
#     2. 특수문자 양쪽에 공백 넣기
#     3. 여러개의 공백은 하나의 공백으로 바꾸기
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
#     5. 다시 양쪽 공백을 지우기
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 필터링 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [12]:
# 정제된 문장 모으기
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인
corpus[:10]

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

In [15]:
#토큰화
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences 사용
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들 것임
    # 우리는 이미 문장을 정제했으니 filters가 필요없음
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀 것임
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus) # 리스트의 형태로 변환 
    tensor = tokenizer.texts_to_sequences(corpus)  #숫자의 시퀀스 형태로 변환
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춤
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2701 2584 ...    0    0    0]
 [   2    7  156 ...    0    0    0]
 [   2   17    7 ...    0    0    0]
 ...
 [   2  311    1 ...    0    0    0]
 [   5   34   45 ... 1161  143    3]
 [   5   34   45 ... 1161  143    3]] <keras_preprocessing.text.Tokenizer object at 0x7ffb5416f0d0>


In [16]:
print(tensor[:3, :10])

[[   2 2701 2584    3    0    0    0    0    0    0]
 [   2    7  156   62  199    3    0    0    0    0]
 [   2   17    7   34    7 1518   15   76    3    0]]


In [17]:
# 단어 사전 만들기
# 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


## (3) 평가 데이터셋 분리

In [18]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 2701 2584    3    0    0    0    0    0    0    0    0    0    0]
[2701 2584    3    0    0    0    0    0    0    0    0    0    0    0]


In [19]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 선언한 대로tokenizer.num_words의 값은 <pad>를 제외한 7000개
VOCAB_SIZE = tokenizer.num_words + 1   

# 데이터셋 제작 
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [21]:
# test, train 분리
from sklearn.model_selection import train_test_split

# 데이터셋을 입력하고, 테스트 세트의 비율을 설정
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)


## (4) 모델 구성 및 학습

In [22]:
# 모델 구성
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 256 # 워드 벡터의 차원수 추상적으로 표현되는 크기
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) 
# pad 때문에 +1추가

In [23]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어 보기
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[ 3.34121461e-04,  3.28293718e-05, -6.17013138e-05, ...,
         -1.52983193e-04,  2.65520357e-04, -1.16119363e-05],
        [ 3.01958440e-04,  2.49672827e-04, -4.17649353e-05, ...,
         -2.82788766e-04,  3.45869747e-04, -1.62855198e-04],
        [ 3.43743479e-04,  5.39079891e-04, -9.35623757e-05, ...,
         -5.42521360e-04,  6.14905730e-04, -5.02781593e-04],
        ...,
        [ 1.72232732e-03,  3.87319044e-04,  3.58989550e-04, ...,
         -1.38842047e-03,  9.69737535e-04,  9.59633733e-04],
        [ 1.94184284e-03,  4.64367389e-04,  1.60526906e-04, ...,
         -1.70378329e-03,  1.04259909e-03,  4.95854532e-04],
        [ 2.13912828e-03,  5.83029003e-04, -4.41790326e-05, ...,
         -2.05494836e-03,  1.10507768e-03,  3.49157472e-06]],

       [[ 3.34121461e-04,  3.28293718e-05, -6.17013138e-05, ...,
         -1.52983193e-04,  2.65520357e-04, -1.16119363e-05],
        [ 5.05282835e-04,  3.70739115e-04,  4.

In [24]:
# 모델의 구조를 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 모델 컴파일과 학습
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, # 기본값은 False, True = softmax함수가 적용되지 않음 
    reduction='none'  # 기본값은 SUM. 각자 나오는 값의 반환 원할 때 None을 사용
)
# 모델 컴파일
model.compile(loss=loss, optimizer=optimizer) 

In [28]:
# 모델 학습
model.fit(dataset, batch_size = 128, epochs=10) 

Epoch 1/10
686/686 [==============================] - 93s 136ms/step - loss: 2.3749
Epoch 2/10
686/686 [==============================] - 93s 135ms/step - loss: 2.2732
Epoch 3/10
686/686 [==============================] - 93s 135ms/step - loss: 2.1775
Epoch 4/10
686/686 [==============================] - 93s 135ms/step - loss: 2.0849
Epoch 5/10
686/686 [==============================] - 93s 135ms/step - loss: 1.9947
Epoch 6/10
686/686 [==============================] - 93s 135ms/step - loss: 1.9065
Epoch 7/10
686/686 [==============================] - 93s 135ms/step - loss: 1.8206
Epoch 8/10
686/686 [==============================] - 93s 135ms/step - loss: 1.7372
Epoch 9/10
686/686 [==============================] - 93s 135ms/step - loss: 1.6565
Epoch 10/10
686/686 [==============================] - 93s 135ms/step - loss: 1.5796


## (5) 평가하기

In [32]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15): # 디폴트값 <start>
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [46]:
generate_text(model, tokenizer, init_sentence="<start> I want") 

'<start> i want to love you p . y . t . <end> '

In [45]:
generate_text(model, tokenizer, init_sentence="<start> I like") 

'<start> i like the way how you re kissin me <end> '

In [47]:
generate_text(model, tokenizer, init_sentence="<start> I love") 

'<start> i love you , liberian girl , <end> '

In [51]:
generate_text(model, tokenizer, init_sentence="<start> I hope") 

'<start> i hope you can take it <end> '

In [50]:
generate_text(model, tokenizer, init_sentence="<start> I just") 

'<start> i just can t stop loving you <end> '

In [52]:
generate_text(model, tokenizer, init_sentence="<start> I hate") 

'<start> i hate the headlines and the weather <end> '

In [54]:
generate_text(model, tokenizer, init_sentence="<start> bad") 

'<start> bad bad really , really bad <end> '

In [56]:
generate_text(model, tokenizer, init_sentence="<start> I m sad") 

'<start> i m sad , yeah , yeah , yeah <end> '

In [57]:
generate_text(model, tokenizer, init_sentence="<start> you look") 

'<start> you look so good <end> '

In [60]:
generate_text(model, tokenizer, init_sentence="<start> we") 

'<start> we re gonna have to say goodbye <end> '

-> 전반적으로 가사에 쓰기에 자연스러운 문장들이 잘 만들어졌다. 

# 프로젝트 회고

## 배운점
1. strip() 함수는 문자열에서 양쪽 끝에 있는 공백이나 특정 문자를 제거하는 데 사용된다.  
2. tf.keras.preprocessing.text.Tokenizer를 사용하면 쉽게 토큰화 할 수 있다. 

## 아쉬운점

## 종합평가

# Reference
1. model.fit 가이드  
https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit  